In [1]:
# Imports local
import sys
sys.path.insert(0, "../")

In [ ]:
#Imports from pip
!pip install git+https://github.com/esgantivar/qmc.git@tn -q

In [7]:
import tensorflow as tf
import numpy as np
from qmc.tn.models import MPSClassifier
from qmc.tn.layers import QFeatureMapRFFTN, MPSLayer
from qmc.tn.features_map import binary_map

In [28]:
#Extract the MNIST Images and preprocess
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
# apply feature map
x_train = binary_map(x_train)
x_test = binary_map(x_test)
# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)
# Y one hot
# y_train = y_train[:, np.newaxis]
# y_test = y_test[:, np.newaxis]
x_test.shape, y_train.shape, x_train.shape, y_test.shape

((10000, 784, 2), (60000, 10), (60000, 784, 2), (10000, 10))

In [25]:
model = MPSClassifier(d_bond=2)

In [29]:
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

In [30]:
history = model.fit(
    x_train,
    y_train,
    epochs=2,
    verbose=1,
    batch_size=1,
    validation_data=(x_test, y_test)
)

Epoch 1/2
1875/1875 [==============================] - 24s 12ms/step - loss: 1.3482 - accuracy: 0.5188 - val_loss: 0.9038 - val_accuracy: 0.7225
Epoch 2/2
1875/1875 [==============================] - 20s 11ms/step - loss: 0.9365 - accuracy: 0.7055 - val_loss: 1.1678 - val_accuracy: 0.6021


In [32]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 2s 5ms/step - loss: 1.1678 - accuracy: 0.6021


[1.1677708625793457, 0.6021000146865845]

## MPS RFF


In [21]:
#Extract the MNIST Images and preprocess
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)
x_test.shape, y_train.shape, x_train.shape, y_test.shape

((10000, 28, 28), (60000, 10), (60000, 28, 28), (10000, 10))

In [22]:
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1] * x_train.shape[2])) / 255.
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1] * x_test.shape[2])) / 255.

In [18]:
x_train.shape, x_test.shape

((60000, 784), (10000, 784))

In [45]:
class MPS(tf.keras.Model):
    def __init__(self, n_sites=28 ** 2, d_bond=2, n_output=10, dim=2, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.mps = MPSLayer(n_sites=n_sites, d_bond=d_bond, n_output=n_output, dim=dim)
        self.fm = QFeatureMapRFFTN(input_dim=n_sites, dim=n_sites, gamma=0.0001)
        self.softmax = tf.keras.layers.Softmax()
    
    def call(self, inputs, **kwargs):
        psi_x = self.fm(inputs)
        print(inputs.shape, psi_x.shape)
        return self.softmax(self.mps(psi_x))

    def get_config(self):
        return super(MPS, self).get_config()

In [46]:
model = MPS(d_bond=2)

In [47]:
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

In [48]:
history = model.fit(
    x_train,
    y_train,
    epochs=2,
    verbose=1,
    batch_size=1,
    validation_data=(x_test, y_test)
)

Epoch 1/2
(1, 784) (1, 784, 2)
(1, 784) (1, 784, 2)
 2049/60000 [>.............................] - ETA: 3:12 - loss: 2.3026 - accuracy: 0.1020

KeyboardInterrupt: 